### Broadband Customer Churn Prediction
Customer churn refers to the loss of existing clients or customers. This solution identifies Broadband customers who are more likely to close their account and leave the Broadband Service. During the training stage, the solution automatically conducts feature interaction on the training data and selects a subset of features based on feature importance. It then trains multiple models and identifies the best performing model. This model is then selected for prediction on new data.

### Contents

1. [Set up the environment](#Set-up-the-environment)
1. [Usage Instructions](#Usage-Instructions)
1. [Upload the data for training](#Upload-the-data-for-training)
1. [Run Training Job](#Run-Training-Job)
1. [Live Inference Endpoint](#Live Inference)
1. [Batch Transform Job](#Batch-Transform-Job)
1. [Output Interpretation](#Output-Interpretation)



<img src="images/Flow_diagram.JPG">

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

### Input format
#### Input:
Name of the file: <b>train.csv</b><br>
This file contains historical churn prediction data for Broadband<br><br>

</ul>
<li>  All Continuous or Categorical variables with any column name can be the input</li>
<li> The target variable should be named 'Churn'</li>

    






## Set up the environment
Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [1]:
# S3 prefix
prefix = 'churn-broadband'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session
The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [2]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training
When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using classification dataset, which we have included.

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [3]:
data_location= 's3://mphasis-marketplace/churn-prediction-broadband/input/train.csv'

## Create an estimator and fit the model
In order to use SageMaker to fit our algorithm, we'll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:
- The container name. This is constructed as in the shell commands above.
- The role. As defined above.
- The instance count which is the number of machines to use for training.
- The instance type which is the type of machine to use for training.
- The output path determines where the model artifact will be written.
- The session is the SageMaker session object that we defined above

Then we use fit() on the estimator to train against the data that we uploaded above.

In [4]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/telecom-pycaret-churn'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 3, 'ml.c4.2xlarge',
                      output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

2021-05-05 13:10:08 Starting - Starting the training job...
2021-05-05 13:10:31 Starting - Launching requested ML instancesProfilerReport-1620220207: InProgress
......
2021-05-05 13:11:31 Starting - Preparing the instances for training......
2021-05-05 13:12:31 Downloading - Downloading input data
2021-05-05 13:12:31 Training - Downloading the training image......
2021-05-05 13:13:31 Training - Training image download completed. Training in progress.Starting the training.
(52523, 21)
IntProgress(value=0, description='Processing: ', max=3)
                                                                    
                                                                    
Initiated  . . . . . . . . . . . . . . . . . .              13:13:27
Status     . . . . . . . . . . . . . . . . . .  Loading Dependencies
                                                                           
                                                                           
Initiated  . . . . . . . . 

## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.


In [5]:
training_job_name = tree.latest_training_job.name
attached_tree = sage.estimator.Estimator.attach(training_job_name)



2021-05-05 13:32:35 Starting - Preparing the instances for training
2021-05-05 13:32:35 Downloading - Downloading input data
2021-05-05 13:32:35 Training - Training image download completed. Training in progress.
2021-05-05 13:32:35 Uploading - Uploading generated training model
2021-05-05 13:32:35 Completed - Training job completed



### Deploy the model
Deploying the model to SageMaker hosting just requires a deploy call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [6]:

from sagemaker.predictor import csv_serializer
predictor = attached_tree.deploy(4, 'ml.m4.xlarge', serializer=csv_serializer,endpoint_name='churn-broadband-pycaret')

---------------!

## Choose some data and use it for a prediction


In [7]:
test_data  = 's3://mphasis-marketplace/churn-prediction-broadband/input/test.csv'

data = pd.read_csv(test_data,encoding='ISO-8859–1',header=None)
data

0          1          2          3           4       5   \
0   image  line_stat  bill_cycl  serv_type   serv_code  tenure   
1  201801         AC          1        BBS    PF100FSN      53   
2  201801         AC          1        BBS  VAXMXMFSXX     198   
3  201801         AC          1        BBS    VS030F6N     199   
4  201801         AC          1        BBS    VS100FSN     198   
5  201801         AC          1        BBS    VS100DUN       3   
6  201801         AC          1        BBS    ES100FSR     198   
7  201801         AC          1        BBS    ES100DUN       9   
8  201801         AC          1        BBS    ES100FSN     198   
9  201801         AC          1        BBS    ES100DUR      90   

               6          7                8            9               10  \
0  contract_month  ce_expiry  secured_revenue    bandwidth  term_reas_code   
1              24         -6             3950  100M (FTTO)             NaN   
2              36        -22             1356          10M             NaN   
3              27        -23             1550          30M             NaN   
4              24         21              390         100M             NaN   
5              24         21              390         100M             NaN   
6              24         14              532         100M             NaN   
7              24         20              532         100M             NaN   
8              28          5              528         100M             NaN   
9              28          5              528         100M             NaN   

               11             12                  13                   14  \
0  term_reas_desc  complaint_cnt  with_phone_service  effc_strt_date_year   
1             NaN              0                   N                 2017   
2             NaN              0                   N                 2015   
3             NaN              1                   N                 2015   
4             NaN              0                   N                 2019   
5             NaN              0                   N                 2019   
6             NaN              0                   N                 2019   
7             NaN              0                   N                 2019   
8             NaN              0                   Y                 2018   
9             NaN              0                   Y                 2018   

                     15                  16                  17  \
0  effc_strt_date_month  effc_strt_date_day  effc_end_date_year   
1                     8                  21                2019   
2                     3                  28                2018   
3                    12                  24                2018   
4                     1                  11                2021   
5                    11                  13                2021   
6                     3                  31                2021   
7                    10                  26                2021   
8                     3                  19                2020   
9                     3                  19                2020   

                    18                 19  
0  effc_end_date_month  effc_end_date_day  
1                    8                 20  
2                    3                 27  
3                    3                 23  
4                   10                 31  
5                   12                 11  
6                    3                 30  
7                   10                 25  
8                    7                 18  
9                    7                 18

In [8]:
predictions = predictor.predict(data.values).decode('utf-8')



The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [9]:
print(predictions)

image,line_stat,bill_cycl,serv_type,serv_code,tenure,contract_month,ce_expiry,secured_revenue,bandwidth,term_reas_code,term_reas_desc,complaint_cnt,with_phone_service,effc_strt_date_year,effc_strt_date_month,effc_strt_date_day,effc_end_date_year,effc_end_date_month,effc_end_date_day,Label,Score
201801,AC,1,BBS,PF100FSN,53,24,-6,3950,100M (FTTO),,,0,N,2017,8,21,2019,8,20,Y,0.96
201801,AC,1,BBS,VAXMXMFSXX,198,36,-22,1356,10M,,,0,N,2015,3,28,2018,3,27,Y,0.97
201801,AC,1,BBS,VS030F6N,199,27,-23,1550,30M,,,1,N,2015,12,24,2018,3,23,Y,0.91
201801,AC,1,BBS,VS100FSN,198,24,21,390,100M,,,0,N,2019,1,11,2021,10,31,N,0.99
201801,AC,1,BBS,VS100DUN,3,24,21,390,100M,,,0,N,2019,11,13,2021,12,11,N,0.98
201801,AC,1,BBS,ES100FSR,198,24,14,532,100M,,,0,N,2019,3,31,2021,3,30,N,0.97
201801,AC,1,BBS,ES100DUN,9,24,20,532,100M,,,0,N,2019,10,26,2021,10,25,N,0.97
201801,AC,1,BBS,ES100FSN,198,28,5,528,100M,,,0,Y,2018,3,19,2020,7,18,N,0.96
201801,AC,1,BBS,ES100DUR,90,28,5,528,100M,,,0,Y,2018,3,19,2020,7,18,N,0.95



### Output

Output files contains column predicted Group, which has the predicted class

In [10]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = tree.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path)

In [11]:
transformer.transform(test_data, content_type='text/csv')
transformer.wait()
print("Batch Transform output saved to " + transformer.output_path)

....................................Starting the inference server with 4 workers.
[2021-05-05 13:47:09 +0000] [13] [INFO] Starting gunicorn 20.1.0
[2021-05-05 13:47:09 +0000] [13] [INFO] Listening at: unix:/tmp/gunicorn.sock (13)
[2021-05-05 13:47:09 +0000] [13] [INFO] Using worker: gevent
[2021-05-05 13:47:09 +0000] [16] [INFO] Booting worker with pid: 16
[2021-05-05 13:47:09 +0000] [17] [INFO] Booting worker with pid: 17
[2021-05-05 13:47:09 +0000] [18] [INFO] Booting worker with pid: 18
[2021-05-05 13:47:09 +0000] [23] [INFO] Booting worker with pid: 23
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
169.254.255.130 - - [05/May/2021:13:47:18 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [05/May/2021:13:47:18 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [05/May/2021:13:47:19 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Transforma

#### Inspect the Batch Transform Output in S3

In [12]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = '{}/{}.out'.format(parsed_url.path[1:], "test.csv")



s3_client = sess.boto_session.client('s3')

response = s3_client.get_object(Bucket = sess.default_bucket(), Key = file_key)
response_bytes = response['Body'].read().decode('utf-8')
print(response_bytes)

image,line_stat,bill_cycl,serv_type,serv_code,tenure,contract_month,ce_expiry,secured_revenue,bandwidth,term_reas_code,term_reas_desc,complaint_cnt,with_phone_service,effc_strt_date_year,effc_strt_date_month,effc_strt_date_day,effc_end_date_year,effc_end_date_month,effc_end_date_day,Label,Score
201801,AC,1,BBS,PF100FSN,53,24,-6,3950,100M (FTTO),,,0,N,2017,8,21,2019,8,20,Y,0.96
201801,AC,1,BBS,VAXMXMFSXX,198,36,-22,1356,10M,,,0,N,2015,3,28,2018,3,27,Y,0.97
201801,AC,1,BBS,VS030F6N,199,27,-23,1550,30M,,,1,N,2015,12,24,2018,3,23,Y,0.91
201801,AC,1,BBS,VS100FSN,198,24,21,390,100M,,,0,N,2019,1,11,2021,10,31,N,0.99
201801,AC,1,BBS,VS100DUN,3,24,21,390,100M,,,0,N,2019,11,13,2021,12,11,N,0.98
201801,AC,1,BBS,ES100FSR,198,24,14,532,100M,,,0,N,2019,3,31,2021,3,30,N,0.97
201801,AC,1,BBS,ES100DUN,9,24,20,532,100M,,,0,N,2019,10,26,2021,10,25,N,0.97
201801,AC,1,BBS,ES100FSN,198,28,5,528,100M,,,0,Y,2018,3,19,2020,7,18,N,0.96
201801,AC,1,BBS,ES100DUR,90,28,5,528,100M,,,0,Y,2018,3,19,2020,7,18,N,0.95



### View Output
Lets read results of above transform job from s3 files and print output

In [13]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/test.csv.out".format(transform_output_folder), '/tmp/test.csv.out')
with open('/tmp/test.csv.out') as f:
    results = f.readlines() 
##print("Transform results: \n{}".format(''.join(results)))
string_final = ''.join(results)

print(string_final)

with open("Output.txt", "w") as text_file:
    text_file.write(string_final)

image,line_stat,bill_cycl,serv_type,serv_code,tenure,contract_month,ce_expiry,secured_revenue,bandwidth,term_reas_code,term_reas_desc,complaint_cnt,with_phone_service,effc_strt_date_year,effc_strt_date_month,effc_strt_date_day,effc_end_date_year,effc_end_date_month,effc_end_date_day,Label,Score
201801,AC,1,BBS,PF100FSN,53,24,-6,3950,100M (FTTO),,,0,N,2017,8,21,2019,8,20,Y,0.96
201801,AC,1,BBS,VAXMXMFSXX,198,36,-22,1356,10M,,,0,N,2015,3,28,2018,3,27,Y,0.97
201801,AC,1,BBS,VS030F6N,199,27,-23,1550,30M,,,1,N,2015,12,24,2018,3,23,Y,0.91
201801,AC,1,BBS,VS100FSN,198,24,21,390,100M,,,0,N,2019,1,11,2021,10,31,N,0.99
201801,AC,1,BBS,VS100DUN,3,24,21,390,100M,,,0,N,2019,11,13,2021,12,11,N,0.98
201801,AC,1,BBS,ES100FSR,198,24,14,532,100M,,,0,N,2019,3,31,2021,3,30,N,0.97
201801,AC,1,BBS,ES100DUN,9,24,20,532,100M,,,0,N,2019,10,26,2021,10,25,N,0.97
201801,AC,1,BBS,ES100FSN,198,28,5,528,100M,,,0,Y,2018,3,19,2020,7,18,N,0.96
201801,AC,1,BBS,ES100DUR,90,28,5,528,100M,,,0,Y,2018,3,19,2020,7,18,N,0.95

